In [ ]:
!pip install ultralytics
!pip install supervision

In [ ]:
from ultralytics import YOLO
import supervision as sv
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cdist
import math
from PIL import Image
from skimage.morphology import remove_small_objects
from skimage.io import imread, imsave
import time
import pandas as pd

In [ ]:
#Modelo de detección (Crear el bounding box para eliminar fondo de la imagen)
model_det = YOLO('/content/drive/MyDrive/DeteccionCircular/best.pt')
#Modelo de segmentación (2 Clases)
model_seg = YOLO('/content/drive/MyDrive/Nuevo_Entrenamientos_PBacterias/best_SinTF_50T.pt')

In [ ]:
def distancia_minima(puntos_contorno_bx, puntos_contorno_by, puntos_contorno_sx, puntos_contorno_sy):
    puntos_b = np.column_stack((puntos_contorno_bx, puntos_contorno_by))
    puntos_s = np.column_stack((puntos_contorno_sx, puntos_contorno_sy))

    distancias = cdist(puntos_b, puntos_s, metric = 'euclidean')
    distancia_min = np.min(distancias)

    indices_min = np.where(distancias == distancia_min)
    x1min, x2min = indices_min[0][0], indices_min[1][0]

    return distancia_min, x1min, x2min

In [ ]:
#Imagenes que no se segmentaron correctamente
errores = []
# Name of the images
nombre_imagenes = []
#Distance pixeles
Distancia_pixeles = []
#Distance mm
Distancia_mm = []
factor = 1295 #Diameter of the visible region of the microscope in pixeles
dir = '/content/Dataset_Prueba50T'
for image in os.listdir(dir):
  path = os.path.join(dir, image)
  print(path)
  img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
  result_det = model_det(path)
  result_det = result_det[0]
  bx, by, bw, bh = np.array(result_det.boxes.xyxy[0].cpu())
  height, width, _ = img.shape
  if((bx+bw)<= (by+bh)):
    radio_bb = int((bx+bw)/2)
  else:
    radio_bb = int((by+bh)/2)
  cx_b, cy_b = int((bx+bw)/2), int((by+bh)/2)
  mask = np.zeros((height, width), dtype = np.uint8)
  mask = cv2.circle(mask, (cx_b, cy_b), radio_bb - 50, 255, -1)
  cv2.imwrite('/content/mask.png', mask)
  mask = cv2.imread('/content/mask.png')
  img2 = cv2.bitwise_and(mask, img)
  #Pasamos la imagen a escala de grises para umbralizarla (binarizarla)
  img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
  umbral = 25
  _ , img2 = cv2.threshold(img2, umbral ,255, cv2.THRESH_BINARY)
  struct = np.ones((5, 5), np.uint8)
  img2 = cv2.morphologyEx(img2, cv2.MORPH_OPEN, struct)
  img2 = cv2.dilate(img2,struct,iterations = 15)
  cv2.imwrite('/content/imgsinFondo.png', img2)
  mask = cv2.imread('/content/imgsinFondo.png')
  clean_img = cv2.bitwise_and(img, mask)
  cv2.imwrite('/content/clean_img.png', clean_img)
  plt.imshow(clean_img); plt.axis('off'); plt.show()
  print(image)

  #____________________________________________________________________________________________________________________________________________________________________________
  result = model_seg.predict('/content/clean_img.png', iou = 0.6, max_det = 2, save = True, show_conf = False,
                             show_labels = True)
  detections = sv.Detections.from_ultralytics(result[0])
  detections_bact = detections[detections.class_id == 0] #ClassID de las bacterias
  detections_sens = detections [detections.class_id == 1] #ClassID del sensidisco
  if(len(detections_sens.mask) != 0 and len(detections_bact.mask) != 0 ):
    nombre_imagenes.append(image)
    mask_bact = np.reshape(detections_bact.mask.flatten(), (height, width)).astype(int)
    mask_sens = np.reshape(detections_sens.mask.flatten(), (height, width)).astype(int)
    mask_bact = np.where(mask_bact, 255, 0).astype(np.uint8)
    mask_sens = np.where(mask_sens, 255, 0).astype(np.uint8)
    plt.gray()
    image1 = Image.fromarray(mask_bact)
    image2 = Image.fromarray(mask_sens)
    image1.save('/content/1.png')
    image2.save('/content/2.png')
    #_______________________________________________________________________________________________
    im = imread('/content/2.png')
    im[im > 0.5] = 1 # create binary image by thresholding with fixed threshold
    im[im <= 0.5] = 0
    im = im.astype(bool)
    im1 = remove_small_objects(im, 40000, connectivity=1)
    imsave('/content/2.png', im1)
    image1 = cv2.cvtColor(cv2.imread('/content/1.png'), cv2.COLOR_BGR2GRAY)
    image2 = cv2.cvtColor(cv2.imread('/content/2.png'), cv2.COLOR_BGR2GRAY)
    umbral = 30
    _ , image1 = cv2.threshold(image1, umbral ,255, cv2.THRESH_BINARY)
    _ , image2 = cv2.threshold(image2, umbral ,255, cv2.THRESH_BINARY)
    contornosimg1, _ = cv2.findContours(image1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contornosimg2, _ = cv2.findContours(image2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    puntos_contorno_bx = []
    puntos_contorno_by = []
    puntos_contorno_sx = []
    puntos_contorno_sy = []
    # Recorrer todos los contornos encontrados para bacterias
    for contorno in contornosimg1:
        for punto in contorno:
            x, y = punto[0]
            puntos_contorno_bx.append(x)
            puntos_contorno_by.append(y)
    # Recorrer todos los contornos encontrados para sensidisco
    for contorno in contornosimg2:
        for punto in contorno:
            x, y = punto[0]
            puntos_contorno_sx.append(x)
            puntos_contorno_sy.append(y)
    masks = cv2.bitwise_or(image1, image2)
    aux, x1min, x2min = distancia_minima(puntos_contorno_bx, puntos_contorno_by, puntos_contorno_sx, puntos_contorno_sy)
    cv2.line(clean_img, (puntos_contorno_bx[x1min], puntos_contorno_by[x1min]),
            (puntos_contorno_sx[x2min], puntos_contorno_sy[x2min]),
            (100, 149, 237), 20)
    plt.imshow(clean_img); plt.axis('off')
    plt.scatter(puntos_contorno_bx[x1min], puntos_contorno_by[x1min], color = 'deeppink', s = 20)
    plt.scatter(puntos_contorno_sx[x2min], puntos_contorno_sy[x2min], color = 'deeppink', s = 20)
    distanciapix_sb = aux
    Distancia_pixeles.append(distanciapix_sb)
    distanciamm_sb = (distanciapix_sb*2) / factor
    Distancia_mm.append(distanciamm_sb)
    plt.title('La distancia es de: ' + str(np.round(distanciamm_sb, 3)) + 'mm')
    plt.show()
  else:
    errores.append(image)
print(errores)

/content/Dataset_Prueba50T/IMG_5073.JPEG



AttributeError: partially initialized module 'torch._dynamo' has no attribute 'trace_rules' (most likely due to a circular import)